# Zillow Prize Challenge

### Import some libraries

In [14]:
import pandas as pd
import numpy as np
import pdb
import seaborn
import math
import seaborn as sns 
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn import preprocessing
import datetime as dt
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy.spatial.distance import pdist
from sklearn.metrics.pairwise import euclidean_distances
import sklearn.metrics.pairwise
import scipy.spatial.distance


%matplotlib inline
warnings.filterwarnings('ignore')


### Read the data from properties, training and submission file(used for training and prediction)

In [3]:
df = pd.read_csv('/Users/kiranmayi/Desktop/Sem1/DSF_519/Hw2/properties_2017.csv', low_memory=False )
#trainingData = pd.read_csv('/Users/kiranmayi/Desktop/Sem1/DSF_519/Hw2/train_2016_v2.csv', low_memory=False )
#submission_data = pd.read_csv('/Users/kiranmayi/Desktop/Sem1/DSF_519/Hw2/sample_submission.csv', low_memory=False )

### Identify which features need Mode and Mean to fill in their missing data

In [4]:
featuresWithMode = ['bathroomcnt',
                    'bedroomcnt',
                    'fireplacecnt',
                    'garagecarcnt',
                    'numberofstories',
                    'unitcnt',
                    'yearbuilt',
                    'regionidzip',
                    'latitude',
                    'longitude']

featuresWithMean = ['finishedsquarefeet12',
                    'finishedsquarefeet15',
                    'calculatedfinishedsquarefeet',
                    'structuretaxvaluedollarcnt',
                    'landtaxvaluedollarcnt',
                    'taxamount']

TotalFeatures = ['parcelid',
                 'bathroomcnt',
                 'bedroomcnt',
                 'fireplacecnt',
                 'garagecarcnt',
                 'numberofstories',
                 'unitcnt',
                 'yearbuilt',
                 'regionidzip',
                 'finishedsquarefeet12',
                 'finishedsquarefeet15',
                 'calculatedfinishedsquarefeet',
                 'structuretaxvaluedollarcnt',
                 'landtaxvaluedollarcnt',
                 'taxamount',
                 'latitude',
                 'longitude']


### Fill NA rows/columns with appropriate Mean or Mode and get the data of interest

In [13]:
cleanedData = deepcopy( df )
for feature in featuresWithMean:
    mean = cleanedData[feature].mean()
    print("Mean for feature ", feature, "is ", mean.real)
    cleanedData[feature].fillna(mean.real, inplace=True)

for feature in featuresWithMode:
    mode = cleanedData[feature].mode()
    print("Mode for feature ", feature, "is ", mode.real[0])
    cleanedData[feature].fillna(mode.real[0], inplace=True)
    
DataOfInterest = cleanedData[ TotalFeatures ]

print(DataOfInterest.head(1))

Mean for feature  finishedsquarefeet12 is  1764.0449917781111
Mean for feature  finishedsquarefeet15 is  2754.8692325893207
Mean for feature  calculatedfinishedsquarefeet is  1831.4552923009944
Mean for feature  structuretaxvaluedollarcnt is  178142.88795247508
Mean for feature  landtaxvaluedollarcnt is  268455.76912108914
Mean for feature  taxamount is  5408.948842055877
Mode for feature  bathroomcnt is  2.0
Mode for feature  bedroomcnt is  3.0
Mode for feature  fireplacecnt is  1.0
Mode for feature  garagecarcnt is  2.0
Mode for feature  numberofstories is  1.0
Mode for feature  unitcnt is  1.0
Mode for feature  yearbuilt is  1955.0
Mode for feature  regionidzip is  96987.0
Mode for feature  latitude is  33984600.0
Mode for feature  longitude is  -118390000.0
   parcelid  bathroomcnt  bedroomcnt  fireplacecnt  garagecarcnt  \
0  10754147          0.0         0.0           1.0           2.0   

   numberofstories  unitcnt  yearbuilt  regionidzip  finishedsquarefeet12  \
0             